In [1]:
from dask.distributed import Client, LocalCluster
from nwm import NWM
from tqdm import tqdm
import pandas as pd
import os
import sys
import logging

In [2]:
start_date_str = "2000-01-01"
end_date_str = "2022-12-31"

comids_file = os.path.join("data", "comid_huc8_boundary_conditions.csv")

comids_hucs = pd.read_csv(comids_file)
comids_hucs

,comid,huc8_from,huc8_to
0,721640,1010002,01010008
1,816963,1010010,
2,817437,1010005,
3,818525,1010011,
4,1025266,1030001,01030003
...,...,...,...
2107,948090631,18090203,
2108,948090648,18090202,18090203
2109,948091000,18090204,
2110,948091293,18090207,


In [3]:
n_workers = 10
threads_per_worker = 2
processes = False

In [4]:
scheduler = LocalCluster(n_workers=n_workers, threads_per_worker=threads_per_worker, processes=processes)

In [5]:
test_comids = comids_hucs.iloc[3:5].comid.to_list()
test_comids

[818525, 1025266]

In [6]:
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO, stream=sys.stdout)


In [7]:
import s3fs
import zarr
import xarray as xr
import numpy as np
import dask
import datetime

NWM_URL = "s3://noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr"
NWM_VARIABLES = ["streamflow", "velocity"]

start_date = datetime.datetime.strptime(start_date_str, "%Y-%m-%d")
end_date = datetime.datetime.strptime(end_date_str, "%Y-%m-%d")

client = Client(scheduler)
client

2023-01-11 09:00:49,286 | INFO : Receive client connection: Client-5a43bf5a-91b8-11ed-bb34-08d23e7f709b
2023-01-11 09:00:49,294 | INFO : Starting established connection to inproc://192.168.1.65/310068/24


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://192.168.1.65:8787/status,
Dashboard: http://192.168.1.65:8787/status,Workers: 10
Total threads: 20,Total memory: 127.76 GiB
Status: running,Using processes: False
Comm: inproc://192.168.1.65/310068/1,Workers: 10
Dashboard: http://192.168.1.65:8787/status,Total threads: 20
Started: Just now,Total memory: 127.76 GiB
Comm: inproc://192.168.1.65/310068/4,Total threads: 2
Dashboard: http://192.168.1.65:62286/status,Memory: 12.78 GiB
Nanny: None,


In [8]:
%%time
s3 = s3fs.S3FileSystem(anon=True)
store = s3fs.S3Map(root=NWM_URL, s3=s3, check=False)
ds = xr.open_zarr(store=store, consolidated=True)
ds

CPU times: total: 11.2 s
Wall time: 42.7 s


<xarray.Dataset>
Dimensions:     (feature_id: 2776738, time: 367439)
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
  * feature_id  (feature_id) int32 101 179 181 ... 1180001803 1180001804
    gage_id     (feature_id) |S15 dask.array<chunksize=(2776738,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(2776738,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T23:0...
Data variables:
    crs         |S1 ...
    streamflow  (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    velocity    (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...

In [9]:
%%time
valid_comids = []
invalid_comids = []
exceptions = {}
found_comids = {}
for comid in comids_hucs.comid.to_list():
    try:
        _comid_test = ds.sel(feature_id=comid, method='nearest')
        found_comid = int(_comid_test.feature_id)
        valid_comids.append(comid)
        if found_comid != comid:
            found_comids[comid] = found_comid
    except Exception as e:
        exceptions[comid] = e
        invalid_comids.append(comid)
print(f"Valid COMIDs: {len(valid_comids)}, Invalid COMIDs: {len(invalid_comids)}, Nearest COMIDs: {len(found_comids)}")

Valid COMIDs: 2112, Invalid COMIDs: 0, Nearest COMIDs: 22
CPU times: total: 41.3 s
Wall time: 48.5 s


In [10]:
selected_comids = valid_comids[0:2]
selected_comids

[721640, 816963]

In [11]:
%%time
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ds_feature = ds[NWM_VARIABLES].sel(
        time=slice(f"{start_date.year}-{start_date.month}-{start_date.day}", f"{end_date.year}-{end_date.month}-{end_date.day}")
    ).sel(
        feature_id=selected_comids, method='nearest'
    ).load(optimize_graph=True, traverse=False)

2023-01-11 09:04:15,949 | INFO : full garbage collection released 47.71 MiB from 1377 reference cycles (threshold: 9.54 MiB)
CPU times: total: 6min 56s
Wall time: 4min 14s


In [12]:
ds_feature

<xarray.Dataset>
Dimensions:     (time: 184104, feature_id: 2)
Coordinates:
    elevation   (feature_id) float32 192.6 98.98
  * feature_id  (feature_id) int32 721640 816965
    gage_id     (feature_id) |S15 b'       01011000' b'               '
    latitude    (feature_id) float32 47.07 46.45
    longitude   (feature_id) float32 -69.07 -67.79
    order       (feature_id) int32 5 2
  * time        (time) datetime64[ns] 2000-01-01 ... 2020-12-31T23:00:00
Data variables:
    streamflow  (time, feature_id) float64 19.38 1.27 19.33 ... 4.89 69.13 4.85
    velocity    (time, feature_id) float64 0.77 0.57 0.77 0.57 ... 0.91 1.36 0.9
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...

In [14]:
ds_df = ds_feature.to_dataframe()
ds_df

streamflow  velocity   elevation  \
time                feature_id                                     
2000-01-01 00:00:00 721640       19.380000      0.77  192.559998   
                    816965        1.270000      0.57   98.980003   
2000-01-01 01:00:00 721640       19.330000      0.77  192.559998   
                    816965        1.270000      0.57   98.980003   
2000-01-01 02:00:00 721640       19.250000      0.77  192.559998   
...                                    ...       ...         ...   
2020-12-31 21:00:00 816965        4.920000      0.91   98.980003   
2020-12-31 22:00:00 721640       69.359998      1.36  192.559998   
                    816965        4.890000      0.91   98.980003   
2020-12-31 23:00:00 721640       69.129998      1.36  192.559998   
                    816965        4.850000      0.90   98.980003   

                                           gage_id   latitude  longitude  \
time                feature_id                                             
2000-01-01 00:00:00 721640      b'       01011000'  47.073555 -69.068634   
                    816965      b'               '  46.448925 -67.787590   
2000-01-01 01:00:00 721640      b'       01011000'  47.073555 -69.068634   
                    816965      b'               '  46.448925 -67.787590   
2000-01-01 02:00:00 721640      b'       01011000'  47.073555 -69.068634   
...                                            ...        ...        ...   
2020-12-31 21:00:00 816965      b'               '  46.448925 -67.787590   
2020-12-31 22:00:00 721640      b'       01011000'  47.073555 -69.068634   
                    816965      b'               '  46.448925 -67.787590   
2020-12-31 23:00:00 721640      b'       01011000'  47.073555 -69.068634   
                    816965      b'               '  46.448925 -67.787590   

                                order  
time                feature_id         
2000-01-01 00:00:00 721640          5  
                    816965          2  
2000-01-01 01:00:00 721640          5  
                    816965          2  
2000-01-01 02:00:00 721640          5  
...                               ...  
2020-12-31 21:00:00 816965          2  
2020-12-31 22:00:00 721640          5  
                    816965          2  
2020-12-31 23:00:00 721640          5  
                    816965          2  

[368208 rows x 7 columns]

In [31]:
comid_df = ds_df.query(f"feature_id=={selected_comids[0]}")
comid_df

,,streamflow,velocity,elevation,gage_id,latitude,longitude,order
time,feature_id,,,,,,,
2000-01-01 00:00:00,721640,19.380000,0.77,192.559998,b' 01011000',47.073555,-69.068634,5
2000-01-01 01:00:00,721640,19.330000,0.77,192.559998,b' 01011000',47.073555,-69.068634,5
2000-01-01 02:00:00,721640,19.250000,0.77,192.559998,b' 01011000',47.073555,-69.068634,5
2000-01-01 03:00:00,721640,19.220000,0.77,192.559998,b' 01011000',47.073555,-69.068634,5
2000-01-01 04:00:00,721640,19.220000,0.77,192.559998,b' 01011000',47.073555,-69.068634,5
...,...,...,...,...,...,...,...,...
2020-12-31 19:00:00,721640,70.019998,1.36,192.559998,b' 01011000',47.073555,-69.068634,5
2020-12-31 20:00:00,721640,69.759998,1.36,192.559998,b' 01011000',47.073555,-69.068634,5
2020-12-31 21:00:00,721640,69.509998,1.36,192.559998,b' 01011000',47.073555,-69.068634,5


In [33]:
comid_csv = comid_df.to_csv()

In [39]:
import sys
sys.getsizeof(comid_csv)

63087750

In [40]:
comid_json = comid_df.to_json()
sys.getsizeof(comid_json)

74697725

In [48]:
min_comid_df = comid_df[["streamflow", "velocity"]]
min_comid_df

,,streamflow,velocity
time,feature_id,,
2000-01-01 00:00:00,721640,19.380000,0.77
2000-01-01 01:00:00,721640,19.330000,0.77
2000-01-01 02:00:00,721640,19.250000,0.77
2000-01-01 03:00:00,721640,19.220000,0.77
2000-01-01 04:00:00,721640,19.220000,0.77
...,...,...,...
2020-12-31 19:00:00,721640,70.019998,1.36
2020-12-31 20:00:00,721640,69.759998,1.36
2020-12-31 21:00:00,721640,69.509998,1.36


In [49]:
output_test_path = os.path.join("data", f"{selected_comids[0]}.h5")
min_comid_df.to_hdf(output_test_path, key="df", mode='w')

In [50]:
h5_read = pd.read_hdf(output_test_path)
h5_read

,,streamflow,velocity
time,feature_id,,
2000-01-01 00:00:00,721640,19.380000,0.77
2000-01-01 01:00:00,721640,19.330000,0.77
2000-01-01 02:00:00,721640,19.250000,0.77
2000-01-01 03:00:00,721640,19.220000,0.77
2000-01-01 04:00:00,721640,19.220000,0.77
...,...,...,...
2020-12-31 19:00:00,721640,70.019998,1.36
2020-12-31 20:00:00,721640,69.759998,1.36
2020-12-31 21:00:00,721640,69.509998,1.36


In [53]:
import sqlite3
db_file = os.path.join("data", "test-database.sqlite")
conn = sqlite3.connect(db_file)

In [54]:
min_comid_df.to_sql('data', con=conn, if_exists='append')

184104

In [59]:
_comid_df = min_comid_df.reset_index()
_comid_df = _comid_df.set_index(["time"])
_comid_df

,feature_id,streamflow,velocity
time,,,
2000-01-01 00:00:00,721640,19.380000,0.77
2000-01-01 01:00:00,721640,19.330000,0.77
2000-01-01 02:00:00,721640,19.250000,0.77
2000-01-01 03:00:00,721640,19.220000,0.77
2000-01-01 04:00:00,721640,19.220000,0.77
...,...,...,...
2020-12-31 19:00:00,721640,70.019998,1.36
2020-12-31 20:00:00,721640,69.759998,1.36
2020-12-31 21:00:00,721640,69.509998,1.36


In [60]:
%%time
daily_streamflow = _comid_df["streamflow"] .resample("D").sum()
daily_streamflow

CPU times: total: 15.6 ms
Wall time: 19 ms


time
2000-01-01     455.579990
2000-01-02     440.469990
2000-01-03     430.539990
2000-01-04     419.819991
2000-01-05     407.599991
                 ...     
2020-12-27    2119.979953
2020-12-28    2229.829950
2020-12-29    1944.559957
2020-12-30    1844.949959
2020-12-31    1721.569962
Freq: D, Name: streamflow, Length: 7671, dtype: float64

In [61]:
%%time
daily_velocity = _comid_df["velocity"] .resample("D").mean()
daily_velocity

CPU times: total: 15.6 ms
Wall time: 6.01 ms


time
2000-01-01    0.763333
2000-01-02    0.755000
2000-01-03    0.750000
2000-01-04    0.742083
2000-01-05    0.736250
                ...   
2020-12-27    1.417500
2020-12-28    1.430417
2020-12-29    1.396250
2020-12-30    1.384583
2020-12-31    1.368750
Freq: D, Name: velocity, Length: 7671, dtype: float64

In [64]:
min_comid_df.shape[0]

184104

In [75]:
ds_df.index.unique(level=1).to_list()

[721640, 816965]